## 第6章 数据加载、存储与文件格式

In [4]:
import pandas as pd
import numpy as np
np.random.seed(2019)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 读写文本格式的数据

In [2]:
df = pd.read_csv('./examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_table('./examples/ex1.csv', sep = '.')    # sep，指定分隔符

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,"a,b,c,d,message"
0,"1,2,3,4,hello"
1,"5,6,7,8,world"
2,"9,10,11,12,foo"


In [9]:
!cat ./examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [21]:
pd.read_csv('./examples/ex2.csv', header=None) # pandas分配默认的列名

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [20]:
pd.read_csv('./examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])  # 自定义列名

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [22]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./examples/ex2.csv', names=names, index_col='message')  # index_col指定索引

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [23]:
parsed = pd.read_csv('./examples/csv_mindex.csv', index_col=['key1', 'key2']) # 层次化索引
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [14]:
!cat ./examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [24]:
result = pd.read_table('./examples/ex3.txt', sep='\s+') # 传入正则表达式表示分隔符
result

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [25]:
# 使用skiprows跳过文件的第一行、第三行和第四行
!cat ./examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [27]:
pd.read_csv('./examples/ex4.csv', skiprows=[0,2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [28]:
# 缺失值处理
!cat ./examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [29]:
result = pd.read_csv('./examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [30]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [32]:
# na_values 可以用一个列表或集合的字符串表示缺失值
result = pd.read_csv('./examples/ex5.csv', na_values='NULL')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [33]:
# 字典的各列可以使用不同的NA标记值
sentinels = {'message':['foo', 'NA'], 'something':['two']}
pd.read_csv('./examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN
